# データ加工

In [1]:
import pandas as pd
import numpy as np
import os

## 学習データに使うcsvファイルのpathを指定

In [2]:
# 試合の結果データ
match_data_path = './data/timetable_20150701-20220724_ver2.csv'

# 各試合に参加した選手に関する情報
player_data_path = './data/timetable_20150701-20220724_siai.csv'

# 作成するcsvのファイルpath
save_path = './data/master_data.csv'

SyntaxError: EOL while scanning string literal (2815932815.py, line 8)

## データの読み込み

In [ ]:
data_match = pd.read_csv(match_data_path, encoding="shift-jis")
data_player = pd.read_csv(player_data_path, encoding="shift-jis")

## 試合結果のデータの整形(結構時間かかる)

In [ ]:
# 必要な列のみ抽出
data = data_match[['タイトル','日次','レース日','レース場','レース回','天候','風向','風速(m)','波の高さ(cm)',
                   '1着_艇番','1着_選手名','1着_モーター番号',
                   '2着_艇番','2着_選手名','2着_モーター番号',
                   '3着_艇番','3着_選手名','3着_モーター番号',
                   '4着_艇番','4着_選手名','4着_モーター番号',
                   '5着_艇番','5着_選手名','5着_モーター番号',
                   '6着_艇番','6着_選手名','6着_モーター番号']]


# '1着_船番'といったカラムから'1枠_順位'といったカラムの作成を行う（処理時間結構長い）
for i in range(len(data)):
    if i == 0:
        for k in range(1,7):
            data[str(k)+'枠_選手名']=0
            data[str(k)+'枠_モーター番号']=0
            data[str(k)+'枠_順位']=0
    hunaban = [data['1着_艇番'][i],data['2着_艇番'][i],data['3着_艇番'][i],data['4着_艇番'][i],data['5着_艇番'][i],data['6着_艇番'][i]]
    argsort_array = np.argsort(hunaban)
    n = 1
    for j in argsort_array:
        data[str(n)+'枠_選手名'][i] = data[str(j+1)+'着_選手名'][i]
        data[str(n)+'枠_モーター番号'][i] = data[str(j+1)+'着_モーター番号'][i]
        data[str(n)+'枠_順位'][i] = j+1
        n += 1



# 不要なカラムの削除＆日付のデータによるsort
data_drop = data.drop(columns=['1着_艇番','1着_選手名','1着_モーター番号',
                   '2着_艇番','2着_選手名','2着_モーター番号',
                   '3着_艇番','3着_選手名','3着_モーター番号',
                   '4着_艇番','4着_選手名','4着_モーター番号',
                   '5着_艇番','5着_選手名','5着_モーター番号',
                   '6着_艇番','6着_選手名','6着_モーター番号'])
#print(data_drop.isnull().sum())
data_drop = data_drop.dropna(subset=['タイトル'])
data_drop = data_drop.dropna(subset=['1枠_選手名'])
#print(data_drop.isnull().sum())
#print(data_drop['レース日'].dtype)
data_shaped = data_drop
data_shaped['レース日'] = pd.to_datetime(data_shaped['レース日'])
#data_drop.sort_values(by = 'レース日', ascending = True, inplace=True)


data_shaped.sort_values(by = ['レース日','レース場','レース回'], ascending = [True, True, True], inplace=True)
data_shaped['レース場'] = data_shaped['レース場'].str.replace(' ', '').str.replace('　', '')
data_shaped['レース回'] = data_shaped['レース回'].str.replace(' ', '').str.replace('　', '')
data_shaped['1枠_選手名'] = data_shaped['1枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_shaped['2枠_選手名'] = data_shaped['2枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_shaped['3枠_選手名'] = data_shaped['3枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_shaped['4枠_選手名'] = data_shaped['4枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_shaped['5枠_選手名'] = data_shaped['5枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_shaped['6枠_選手名'] = data_shaped['6枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_shaped['レース回'] = data_shaped['レース回'].str.replace('01R', '1').str.replace('02R', '2').str.replace('03R', '3').str.replace('04R', '4').str.replace('05R', '5').str.replace('06R', '6').str.replace('07R', '7').str.replace('08R', '8').str.replace('09R', '9').str.replace('10R', '10').str.replace('11R', '11').str.replace('12R', '12')




## 選手情報のデータの整形

In [ ]:
# 必要な列のみ抽出
data_player = data_match[['レース日','レース場','レース回',
                   '1枠_選手名','1枠_年齢','1枠_支部','1枠_体重','1枠_全国勝率','1枠_全国2連対率','1枠_当地勝率','1枠_当地2連対率',
                   '2枠_選手名','2枠_年齢','2枠_支部','2枠_体重','2枠_全国勝率','2枠_全国2連対率','2枠_当地勝率','2枠_当地2連対率',
                   '3枠_選手名','3枠_年齢','3枠_支部','3枠_体重','3枠_全国勝率','3枠_全国2連対率','3枠_当地勝率','3枠_当地2連対率',
                   '4枠_選手名','4枠_年齢','4枠_支部','4枠_体重','4枠_全国勝率','4枠_全国2連対率','4枠_当地勝率','4枠_当地2連対率',
                   '5枠_選手名','5枠_年齢','5枠_支部','5枠_体重','5枠_全国勝率','5枠_全国2連対率','5枠_当地勝率','5枠_当地2連対率',
                   '6枠_選手名','6枠_年齢','6枠_支部','6枠_体重','6枠_全国勝率','6枠_全国2連対率','6枠_当地勝率','6枠_当地2連対率']]

# レース日のフォーマットの調整
data_player['レース日'] = data_player['レース日'].str.replace(' ', '').str.replace('　', '').str.replace('１','1').str.replace('２','2').str.replace('３','3').str.replace('４','4').str.replace('５','5').str.replace('６','6').str.replace('７','7').str.replace('８','8').str.replace('９','9').str.replace('０','0')
data_player['レース日'] = pd.to_datetime(data_player['レース日'], format='%Y年%m月%d日')

# レース日、レース場、レース回の順でsort
data_player.sort_values(by = ['レース日','レース場','レース回'], ascending = [True, True, True], inplace=True)

# 不要なカラムの削除
data_player.drop(columns=['Unnamed: 0', '1枠_支部', '2枠_支部', '3枠_支部', '4枠_支部', '5枠_支部', '6枠_支部'], inplace=True)


# 試合データとのmergeaのためにデータのフォーマットの統一
data_player['レース場'] = data_player['レース場'].str.replace(' ', '').str.replace('　', '')
data_player['レース回'] = data_player['レース回'].str.replace(' ', '').str.replace('　', '')
data_player['1枠_選手名'] = data_player['1枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_player['2枠_選手名'] = data_player['2枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_player['3枠_選手名'] = data_player['3枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_player['4枠_選手名'] = data_player['4枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_player['5枠_選手名'] = data_player['5枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_player['6枠_選手名'] = data_player['6枠_選手名'].str.replace(' ', '').str.replace('　', '')
data_player['レース回'] = data_player['レース回'].str.replace('１Ｒ', '1').str.replace('２Ｒ', '2').str.replace('３Ｒ', '3').str.replace('４Ｒ', '4').str.replace('５Ｒ', '5').str.replace('６Ｒ', '6').str.replace('７Ｒ', '7').str.replace('８Ｒ', '8').str.replace('９Ｒ', '9').str.replace('１０Ｒ', '10')
data_player['レース回'] = data_player['レース回'].str.replace('１1', '11').str.replace('１2', '12')



## 試合データと選手データのmerge

In [ ]:
# レース日、レース場、レース回をキー値として、試合データと選手データをmergeする
data = pd.merge(data_shaped, data_player, on =['レース日','レース場','レース回'], how='inner', sort=True)

In [ ]:
data['レース日'] = pd.to_datetime(data['レース日'])
data['レース日'] = data['レース日'].dt.strftime('%Y%m%d')

In [ ]:
data.to_csv(save_path, encoding = "shift-jis",index=False)